In [75]:
DATA_PATH = '/content/drive/MyDrive/데이터 분석/데이콘 캐글 컴페티션/예슬 개인 참가/제주 특산물 가격 예측/data/'

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/jeju-dataset/sample_submission.csv
/kaggle/input/jeju-dataset/train.csv
/kaggle/input/jeju-dataset/test.csv
/kaggle/input/jeju-dataset/international_trade.csv


In [54]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn import tree
# from catboost import CatBoostRegressor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from sklearn.model_selection import KFold, train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error, make_scorer

In [76]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

DATA_PATH = '/kaggle/input/jeju-dataset/'

SEED = 42

train = pd.read_csv(f'{DATA_PATH}train.csv')
test = pd.read_csv(f'{DATA_PATH}test.csv')
submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')
trade = pd.read_csv(f'{DATA_PATH}international_trade.csv')

In [77]:

train['name'] = train['ID'].apply(lambda x: x[:6])
test['name'] = test['ID'].apply(lambda x: x[:6])

In [78]:
train.rename(columns={'supply(kg)': 'supply_kg', 'price(원/kg)': 'price'}, inplace=True)
test.rename(columns={'supply(kg)': 'supply_kg', 'price(원/kg)': 'price'}, inplace=True)


## 시간 Feature

In [79]:
train.timestamp = pd.to_datetime(train.timestamp)
test.timestamp = pd.to_datetime(test.timestamp)


In [80]:
train['year'] = train.timestamp.dt.year
train['month'] = train.timestamp.dt.month
train['day'] = train.timestamp.dt.day # 드롭예정
train['day_of_week'] = train.timestamp.dt.dayofweek
train['day_of_year'] = train.timestamp.dt.dayofyear
train['week_of_year'] = train.timestamp.dt.isocalendar().week  # 연중 주간 (ISO 주차 기준)
# train['is_weekend'] = train.day_of_week.apply(lambda x: 1 if x >= 5 else 0)
train['is_sunday'] = train.day_of_week.apply(lambda x: 1 if x == 6 else 0)


In [81]:
test['year'] = test.timestamp.dt.year
test['month'] = test.timestamp.dt.month
test['day'] = test.timestamp.dt.day # 드롭예정
test['day_of_week'] = test.timestamp.dt.dayofweek
test['day_of_year'] = test.timestamp.dt.dayofyear
test['week_of_year'] = test.timestamp.dt.isocalendar().week  # 연중 주간 (ISO 주차 기준)
# test['is_weekend'] = test.day_of_week.apply(lambda x: 1 if x >= 5 else 0)
test['is_sunday'] = test.day_of_week.apply(lambda x: 1 if x == 6 else 0)


In [82]:

# train, test에 추가

price_name_y_m_dow_avg = train.groupby(['name','year','month','day_of_week'])['price'].mean().reset_index()
train = train.merge(price_name_y_m_dow_avg, on = ['name','year','month','day_of_week'],how='left',suffixes=('','_name_y_m_dow_avg'))
test = test.merge(price_name_y_m_dow_avg, on = ['name','year','month','day_of_week'],how='left',suffixes=('','_name_y_m_dow_avg'))

price_name_y_m_dow_median = train.groupby(['name','year','month','day_of_week'])['price'].median().reset_index()
train = train.merge(price_name_y_m_dow_median, on = ['name','year','month','day_of_week'],how='left',suffixes=('','_name_y_m_dow_median'))
test = test.merge(price_name_y_m_dow_median, on = ['name','year','month','day_of_week'],how='left',suffixes=('','_name_y_m_dow_median'))

supply_corp_m_std = train.groupby(['corporation','month'])['supply_kg'].std().reset_index()
train = train.merge(supply_corp_m_std, on = ['corporation','month'],how='left',suffixes=('','_corp_m_std'))
test = test.merge(supply_corp_m_std, on = ['corporation','month'],how='left',suffixes=('','_corp_m_std'))

supply_corp_m_max = train.groupby(['corporation','month'])['supply_kg'].max().reset_index()
train = train.merge(supply_corp_m_max, on = ['corporation','month'],how='left',suffixes=('','_corp_m_max'))
test = test.merge(supply_corp_m_max, on = ['corporation','month'],how='left',suffixes=('','_corp_m_max'))


In [83]:
trade = pd.read_csv(f'{DATA_PATH}international_trade.csv')

In [84]:
trade['기간']= pd.to_datetime(trade['기간'])
trade['year'] = trade['기간'].dt.year
trade['month'] = trade['기간'].dt.month

trade['수출 금액'] = trade['수출 금액']*1000 * 1200 #천달러 단위 -> 환율 1200원으로 계산
trade['수입 금액'] = trade['수입 금액']*1000 * 1200

trade['무역수지'] = trade['수출 금액'] - trade['수입 금액']
trade['수출 단가'] = trade['수출 금액'] / trade['수출 중량']
trade['수입 단가'] = trade['수입 금액'] / trade['수입 중량']

trade = trade.fillna(0)

trade.columns = ['timestamp','item','exp_supply','exp_revenue','imp_supply','imp_revenue','trade_balance','year','month','exp_price','imp_price']

In [85]:
# 방울다다기 양배추 제외 버전
mask = trade['item'].str.contains('감귤|브로콜리|순무|당근|양배추')
trade_items = trade[mask]

trade_items = trade_items.replace({'꽃양배추와 브로콜리(broccoli)':'BC',
                           '양배추':'CB',
                           '당근':'CR',
                           '감귤':'TG',
                           '순무':'RD'})
cb_extra = trade_items['item'] == '방울다다기 양배추' # 방울다다기 양배추 미포함
trade_items = trade_items[~cb_extra]

trade_items = trade_items.drop(columns = 'timestamp').reset_index().drop(columns = 'index')


trade_items_month = pd.pivot_table(trade_items, index=['item','month'],fill_value=0).reset_index()
trade_items_month = trade_items_month.round(1)

# trade_items = pd.pivot_table(trade_items, index=['item','year','month'],fill_value=0).reset_index()
# trade_items = trade_items.round(1)

# * 평균이랑 + max-min평균

In [86]:
trade_items_month

,item,month,exp_price,exp_revenue,exp_supply,imp_price,imp_revenue,imp_supply,trade_balance,year
0,BC,1,9000.0,480000,46.4,1307.3,902160000,682774.2,-901680000,2021.0
1,BC,2,1057.7,1200000,423.2,1277.8,519360000,403745.2,-518160000,2021.0
2,BC,3,24000.0,1200000,45.5,1331.6,983400000,753128.5,-982200000,2020.5
3,BC,4,3571.4,300000,33.5,1503.9,1327200000,890748.0,-1326900000,2020.5
4,BC,5,18750.0,300000,18.0,1422.4,1027500000,722724.2,-1027200000,2020.5
5,BC,6,0.0,0,12.0,1342.4,682800000,513319.5,-682800000,2020.5
6,BC,7,0.0,0,20.0,1391.0,892800000,642013.2,-892800000,2020.5
7,BC,8,0.0,0,8.2,1441.8,1760100000,1224899.8,-1760100000,2020.5
8,BC,9,0.0,0,6.0,1427.8,2012400000,1407729.0,-2012400000,2020.5
9,BC,10,0.0,0,6.0,1376.7,1297200000,971622.5,-1297200000,2020.5


In [87]:
from numpy.core.numeric import tensordot
# 버전 1 ) item, year, month 기준 채우고 3월 값들은 0으로 대체

# train = pd.merge(train,trade_items,on=['item','year','month'],how='left')
# test = pd.merge(test,trade_items,on=['item','year','month'],how='left')

# # 버전 2 ) item, month 기준 채우기
trade_items_month = trade_items_month.drop(columns='year')

train = pd.merge(train,trade_items_month,on=['item','month'],how='left')
test = pd.merge(test,trade_items_month,on=['item','month'],how='left')

In [88]:
train.shape, test.shape

((59397, 26), (1092, 24))

In [89]:
# 선형보간법으로 결측치 채우기 -> 결측치 채우는 방법 확인

train = train.fillna(0)
test = test.fillna(0)

In [90]:
test.isna().sum()

ID                           0
timestamp                    0
item                         0
corporation                  0
location                     0
name                         0
year                         0
month                        0
day                          0
day_of_week                  0
day_of_year                  0
week_of_year                 0
is_sunday                    0
price                        0
price_name_y_m_dow_median    0
supply_kg                    0
supply_kg_corp_m_max         0
exp_price                    0
exp_revenue                  0
exp_supply                   0
imp_price                    0
imp_revenue                  0
imp_supply                   0
trade_balance                0
dtype: int64

In [91]:
train.isna().sum()

ID                           0
timestamp                    0
item                         0
corporation                  0
location                     0
supply_kg                    0
price                        0
name                         0
year                         0
month                        0
day                          0
day_of_week                  0
day_of_year                  0
week_of_year                 0
is_sunday                    0
price_name_y_m_dow_avg       0
price_name_y_m_dow_median    0
supply_kg_corp_m_std         0
supply_kg_corp_m_max         0
exp_price                    0
exp_revenue                  0
exp_supply                   0
imp_price                    0
imp_revenue                  0
imp_supply                   0
trade_balance                0
dtype: int64

## 검증, 예측

In [107]:
train_ft = train.copy()
test_ft = test.copy()


train_ft.shape, test_ft.shape

((59397, 26), (1092, 24))

In [108]:
cat_train = train_ft[['item']]

cat_train_encoded = pd.get_dummies(cat_train)
cat_train_encoded = cat_train_encoded.applymap(lambda x: 1 if x != 0 else 0)

cat_test = test_ft[['item']]

cat_test_encoded = pd.get_dummies(cat_test)
cat_test_encoded = cat_test_encoded.applymap(lambda x: 1 if x != 0 else 0)


train_ft = pd.concat([train_ft,cat_train_encoded],axis=1)
test_ft = pd.concat([test_ft,cat_test_encoded],axis=1)

In [109]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

train_ft['name'] = label_encoder.fit_transform(train_ft['name'])
test_ft['name'] = label_encoder.transform(test_ft['name'])

# train_ft['corporation'] = train_ft['corporation'].replace({'A':0,'B':1,'C':2,'D':3,'E':4,'F':5})
# train_ft['location'] = train_ft['location'].replace({'J':0,'S':1})

# test_ft['corporation'] = test_ft['corporation'].replace({'A':0,'B':1,'C':2,'D':3,'E':4,'F':5})
# test_ft['location'] = test_ft['location'].replace({'J':0,'S':1})

In [110]:
train_ft

,ID,timestamp,item,corporation,location,supply_kg,price,name,year,month,...,exp_supply,imp_price,imp_revenue,imp_supply,trade_balance,item_BC,item_CB,item_CR,item_RD,item_TG
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,29,2019,1,...,192621.6,0.0,0.0,0.2,380880000.0,0,0,0,0,1
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,29,2019,1,...,192621.6,0.0,0.0,0.2,380880000.0,0,0,0,0,1
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,29,2019,1,...,192621.6,0.0,0.0,0.2,380880000.0,0,0,0,0,1
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,29,2019,1,...,192621.6,0.0,0.0,0.2,380880000.0,0,0,0,0,1
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,29,2019,1,...,192621.6,0.0,0.0,0.2,380880000.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,28,2023,2,...,4000.0,0.0,0.0,2.0,4800000.0,0,0,0,1,0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,28,2023,2,...,4000.0,0.0,0.0,2.0,4800000.0,0,0,0,1,0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,28,2023,3,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,28,2023,3,...,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0


In [111]:
drop_cols = ['ID','timestamp','day','year','item','corporation','location',
             'exp_revenue'
            ]

train_ft = train_ft.drop(columns = drop_cols)
test_ft = test_ft.drop(columns = drop_cols)

train_ft.shape, test_ft.shape

((59397, 23), (1092, 21))

In [112]:
train_ft.corr()

,supply_kg,price,name,month,day_of_week,day_of_year,week_of_year,is_sunday,price_name_y_m_dow_avg,price_name_y_m_dow_median,...,exp_supply,imp_price,imp_revenue,imp_supply,trade_balance,item_BC,item_CB,item_CR,item_RD,item_TG
supply_kg,1.000000,-0.002297,1.427750e-01,-1.205799e-01,-6.979763e-02,-1.197653e-01,-1.227554e-01,-9.199102e-02,-0.006934,-0.004917,...,0.057431,-0.125279,-0.097535,-0.075073,0.109673,-1.141741e-01,-4.791316e-03,-6.121449e-02,1.533454e-01,2.582781e-02
price,-0.002297,1.000000,3.706544e-01,-7.667260e-02,-1.526107e-01,-7.596119e-02,-8.089969e-02,-2.264767e-01,0.919521,0.905813,...,-0.070830,0.150529,-0.237897,-0.231997,0.210231,-5.976174e-02,-1.850407e-01,-1.978180e-01,-2.363935e-01,5.918016e-01
name,0.142775,0.370654,1.000000e+00,-6.387365e-15,-2.290579e-15,4.738604e-15,2.287522e-15,-3.765381e-16,0.403095,0.395645,...,0.113721,-0.365633,-0.330857,-0.198081,0.365087,-7.299964e-01,-2.725865e-01,-8.311375e-02,2.482537e-01,7.565503e-01
month,-0.120580,-0.076673,-6.387365e-15,1.000000e+00,-2.460519e-03,9.966339e-01,9.686496e-01,-3.485451e-03,-0.083383,-0.085975,...,0.233748,-0.198340,0.038358,0.059226,0.039935,6.199739e-15,-1.323247e-16,-2.725820e-15,3.691655e-15,-6.887810e-15
day_of_week,-0.069798,-0.152611,-2.290579e-15,-2.460519e-03,1.000000e+00,-2.446467e-03,2.038724e-04,6.122044e-01,-0.165968,-0.159981,...,-0.002929,0.000435,-0.000880,-0.000533,-0.000073,2.804264e-15,-1.121082e-16,-1.782418e-15,1.569668e-15,-2.505554e-15
day_of_year,-0.119765,-0.075961,4.738604e-15,9.966339e-01,-2.446467e-03,1.000000e+00,9.699002e-01,-3.103241e-03,-0.083823,-0.086334,...,0.233076,-0.198446,0.037975,0.058935,0.040164,-5.043272e-15,8.912885e-16,1.335163e-15,-1.900225e-15,4.723285e-15
week_of_year,-0.122755,-0.080900,2.287522e-15,9.686496e-01,2.038724e-04,9.699002e-01,1.000000e+00,1.030839e-03,-0.085485,-0.087665,...,0.227286,-0.201677,0.036053,0.057203,0.040534,-4.911588e-15,1.653998e-15,2.613766e-15,-3.099041e-16,1.462328e-15
is_sunday,-0.091991,-0.226477,-3.765381e-16,-3.485451e-03,6.122044e-01,-3.103241e-03,1.030839e-03,1.000000e+00,-0.246299,-0.237354,...,-0.002040,-0.000231,-0.000878,-0.000813,0.000331,4.530440e-16,1.106054e-16,-7.262455e-16,4.977259e-16,-3.447449e-16
price_name_y_m_dow_avg,-0.006934,0.919521,4.030953e-01,-8.338322e-02,-1.659676e-01,-8.382309e-02,-8.548491e-02,-2.462986e-01,1.000000,0.985092,...,-0.077029,0.163704,-0.258719,-0.252302,0.228631,-6.499228e-02,-2.012360e-01,-2.151316e-01,-2.570834e-01,6.435979e-01
price_name_y_m_dow_median,-0.004917,0.905813,3.956447e-01,-8.597505e-02,-1.599811e-01,-8.633437e-02,-8.766493e-02,-2.373539e-01,0.985092,1.000000,...,-0.076134,0.149728,-0.253060,-0.245550,0.223284,-6.734363e-02,-1.956008e-01,-2.085526e-01,-2.476070e-01,6.270067e-01


# autogluon

In [113]:
train_ft = train_ft.drop(columns = 'supply_kg')
test_ft = test_ft.rename(columns={'price':'price_name_y_m_dow_avg',
                        'supply_kg':'supply_kg_corp_m_std'})

In [114]:
train_ft.shape, test_ft.shape

((59397, 22), (1092, 21))

In [115]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 4.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 62.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 17.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 73.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [116]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [117]:
train_ft.columns

Index(['price', 'name', 'month', 'day_of_week', 'day_of_year', 'week_of_year',
       'is_sunday', 'price_name_y_m_dow_avg', 'price_name_y_m_dow_median',
       'supply_kg_corp_m_std', 'supply_kg_corp_m_max', 'exp_price',
       'exp_supply', 'imp_price', 'imp_revenue', 'imp_supply', 'trade_balance',
       'item_BC', 'item_CB', 'item_CR', 'item_RD', 'item_TG'],
      dtype='object')

In [118]:
train_ft.shape, test_ft.shape

((59397, 22), (1092, 21))

In [119]:
label = 'price'
predictor = TabularPredictor(label=label).fit(train_ft,presets='best_quality' )

# predictor = TabularPredictor(label = label, problem_type = 'regression', eval_metric = 'rmse')
# predictor.fit(train_data = train_ft, presets = 'best_quality',
#               auto_stack = True, fit_weighted_ensemble = True,
#               num_bag_folds = 10, num_bag_sets = 30, num_stack_levels = 3, verbosity = 2)

#,higher_is_better = False


No path specified. Models will be saved in: "AutogluonModels/ag-20231108_223139/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Cannot interpret 'UInt32Dtype()' as a data type
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20231108_223139/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Nov 4 11:53:48 UTC 2023
Disk Space Avail:   20.94 GB / 20.96 GB (99.9%)
Train Data Rows:    59397
Train Data Columns: 21
Label Column: price
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (20909.0, 0.0, 1131.68067, 2029.94145)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (Y

In [124]:
# train_ft.columns
predictor = TabularPredictor.load("AutogluonModels/ag-20231108_223139/")
# /kaggle/working/AutogluonModels 

In [125]:
# test_ft.columns

predictor

In [ ]:
# test_ft = test_ft.fillna(0)

In [126]:
predictions = predictor.predict(test_ft)
predictions.shape

INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU


(1092,)

In [127]:
predictions



0         12.811711
1          6.379787
2         15.729284
3         12.433157
4       3195.165527
           ...     
1087       0.388106
1088      -0.172169
1089     637.849731
1090     539.776306
1091     541.838867
Name: price, Length: 1092, dtype: float32

In [128]:
submission['answer']= predictions
submission

,ID,answer
0,TG_A_J_20230304,12.811711
1,TG_A_J_20230305,6.379787
2,TG_A_J_20230306,15.729284
3,TG_A_J_20230307,12.433157
4,TG_A_J_20230308,3195.165527
...,...,...
1087,RD_F_J_20230327,0.388106
1088,RD_F_J_20230328,-0.172169
1089,RD_F_J_20230329,637.849731
1090,RD_F_J_20230330,539.776306


In [130]:
# submission.to_csv(f'{DATA_PATH}autogluon_ver1.csv', index=False)
submission.to_csv(f'autogluon_ver1.csv', index=False)




In [131]:
modified_prices = np.maximum(predictions, 0)
modified_prices

# submission.to_csv(f'autogluon_ver1.csv', index=False)


0         12.811711
1          6.379787
2         15.729284
3         12.433157
4       3195.165527
           ...     
1087       0.388106
1088       0.000000
1089     637.849731
1090     539.776306
1091     541.838867
Name: price, Length: 1092, dtype: float32

In [132]:
submission['answer'] = modified_prices
submission

,ID,answer
0,TG_A_J_20230304,12.811711
1,TG_A_J_20230305,6.379787
2,TG_A_J_20230306,15.729284
3,TG_A_J_20230307,12.433157
4,TG_A_J_20230308,3195.165527
...,...,...
1087,RD_F_J_20230327,0.388106
1088,RD_F_J_20230328,0.000000
1089,RD_F_J_20230329,637.849731
1090,RD_F_J_20230330,539.776306


In [133]:
submission.to_csv(f'autogluon_ver1_mto0.csv', index=False)


# optuna

In [ ]:
!pip install optuna


In [ ]:
import optuna

In [ ]:
X = train_ft.drop(columns = ['price'])
y = train_ft['price']

test_X = test_ft

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size=0.2, random_state=SEED)

X_train.shape, X_valid.shape, y_train.shape, y_valid.shape, test_X.shape

In [ ]:
model= LGBMRegressor(random_state=SEED)
model.fit(X_train, y_train)
X_pred = model.predict(X_train)
X_rmse = np.sqrt(mean_squared_error(y_train, X_pred))

X_rmse

In [ ]:
x_train = train.drop(columns=['name', 'price'])
y_train = train['price']

x_test  = test.drop(columns=['name'])

In [ ]:
def objective(trial):
    param = {
        'objective': 'regression', # 회귀
        'verbose': -1,
        'metric': 'rmse',
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }

    model = LGBMRegressor(**param)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, preds))

    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
# !pip install mljar-supervised

In [ ]:
# from supervised.automl import AutoML


In [ ]:
x_train = train.drop(columns=['name', 'price'])
y_train = train['price']
x_test  = test.drop(columns=['name'])

In [ ]:
automl = AutoML(mode="Optuna", optuna_time_budget=36)


In [2]:
automl.fit(x_train, y_train)

NameError: name 'automl' is not defined

In [3]:
# X = train_ft.drop(columns = ['price','supply_kg'])
# y = train_ft['price']

# test_X = test_ft

In [4]:
# Split
from sklearn.model_selection import KFold, train_test_split, cross_val_score, StratifiedKFold

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.2, random_state=SEED)

# # Calculate the index to split the data
# split_index = int(0.8 * len(X))

# # Split the data into training and validation sets
# X_train, X_valid = X[:split_index], X[split_index:]
# y_train, y_valid = y[:split_index], y[split_index:]

X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

NameError: name 'X' is not defined

In [ ]:
model = XGBRegressor(random_state=SEED)
model.fit(X_train, y_train)
X_pred = model.predict(X_train)
X_rmse = np.sqrt(mean_squared_error(y_train, X_pred))

X_rmse

# hold 707.3795681078395
# 뒷부분 Hold : 753.6527649085411
# 모든 feature 추가 : 560.0123072446424 (onehot encoding)
# holdout : 507.71805618161835

In [ ]:
model = LGBMRegressor(random_state=SEED)
model.fit(X_train, y_train)
X_pred = model.predict(X_train)
X_rmse = np.sqrt(mean_squared_error(y_train, X_pred))

X_rmse

# 뒷부분 Hold : 843.9928276331788
# ym unix 포함 : 847.5193982992201
# 전체 : 726.5042502409534
# 전체 Holdout : 673.5662950313344

In [ ]:
model = RandomForestRegressor(random_state=SEED)
model.fit(X_train, y_train)
X_pred = model.predict(X_train)
X_rmse = np.sqrt(mean_squared_error(y_train, X_pred))

X_rmse

# 뒷부분 Hold 343.52258075359885
# ym unix 포함 :
# 모든 피쳐 포함 : 330.62296335942256 (원핫인코딩))

In [ ]:
def objective_rf(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'max_depth': trial.suggest_int('max_depth', 3, 50),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'gamma': trial.suggest_float('gamma', 0, 1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        # 'enable_categorical': True,
        # 'tree_method': 'hist'  # 다른 트리 메서드 시도
    }

    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, preds))

    return rmse

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'max_depth': trial.suggest_int('max_depth', 3, 50),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'gamma': trial.suggest_float('gamma', 0, 1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        # 'enable_categorical': True,
        # 'tree_method': 'hist'  # 다른 트리 메서드 시도
    }

    model = XGBRegressor(**params)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    rmse = np.sqrt(mean_squared_error(y_valid, preds))

    return rmse


In [ ]:
!pip install optuna

In [ ]:
!pip install --upgrade optuna


In [ ]:
import optuna

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
# [I 2023-11-08 06:06:49,505] Trial 4 finished with value: 845.238901560459 and parameters: {'n_estimators': 772, 'max_depth': 11, 'learning_rate': 0.06209451206011337, 'gamma': 0.1541323298249201, 'min_child_weight': 10, 'subsample': 0.8877306745243241, 'colsample_bytree': 0.3922321423322589, 'reg_alpha': 9.295858630218872, 'reg_lambda': 2.4971154174509858}. Best is trial 4 with value: 845.238901560459.

In [ ]:
study.best_params

In [ ]:
best_params = study.best_params
best_params

In [ ]:
model = XGBRegressor(**best_params)
model.fit(X_train, y_train)
X_pred = model.predict(X_train)
X_rmse = np.sqrt(mean_squared_error(y_train, X_pred))

X_rmse

In [ ]:
pred = model.predict(test_X)
pred

In [ ]:
pred[pred < 0] = 0

submission['answer'] = pred
submission

In [ ]:
submission.to_csv(f"{DATA_PATH}ml_ver1_xgb_100.csv", index=False)
submission.to_csv("ml_ver1_xgb_100.csv", index=False)

In [ ]:
# 다양한 기준으로, 다양한 피처를 추가할 것. name 뿐이아니라!


train

# 요소
# 시간 : 년, 월, 일, 요일, 연중주, 연중일, 시간? or sin cos (주기)?
# 품목
# 법인
# 지역
# 공급물량
# 수출금액
# 수출물량
# 수입금액
# 수입물량
# 무역수지

import itertools

elements = ['year','month','day','dayofweek','dayofyear','weekofyear',
           'item','corporation','location','supply',
           'imp_supply','imp_sales','exp_supply','exp_sales','trade_balance']  # Replace this with your list of elements

# # Generate all combinations of elements from the list
# all_combinations = []
# for r in range(1, len(elements) + 1):
#     combinations_r = list(itertools.combinations(elements, r))
#     all_combinations.extend(combinations_r)

# # Print all combinations
# print(all_combinations)